In [ ]:
!pip install --upgrade --quiet openai langchain langchainhub tiktoken chromadb langchain-community langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Set OpenAI API Key as an environment variable
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-toMRbOoAEksbB6zhlpJET3BlbkFJtEwHJFoEPmPGMf6D9YU0"

In [ ]:
# Import the required modules
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

(1) WebBaseLoader() 를 이용해서 웹사이트 문서 로딩하기

In [ ]:
# Define basic web pages to load
basic_web_loader = WebBaseLoader(
    web_paths=("https://artsandculture.google.com/asset/self-portrait/9gFw_1Vou2CkwQ", # Self-Portrait
               "https://artsandculture.google.com/asset/the-bedroom-0097/rgHdFPzCeCfnxQ", # The bedroom
               "https://artsandculture.google.com/asset/the-starry-night-vincent-van-gogh/bgEuwDxel93-Pg", # The Starry Night
               "https://artsandculture.google.com/asset/sunflowers-vincent-van-gogh/hwEGmsM-FoHAwA" # Sunflowers
               ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["R5VDUc", "PugfHe"]},
        )
    ),
)

basic_data = basic_web_loader.load()

# Define the additional web pages to load
additional_web_loader = WebBaseLoader(
    web_paths=[
        "https://artsandculture.google.com/story/jgWxmsxoUpN42A", # The Woman Who Made Van Gogh Famous
        "https://artsandculture.google.com/story/oQURDM2pcsofJg", # The Life and Works of Van Gogh
        "https://artsandculture.google.com/story/10-things-you-might-not-know-about-vincent-van-gogh/agLStrYjblBRKA", # 10 Things You Might Not Know About Vincent van Gogh
        "https://artsandculture.google.com/story/FwXR3IqVOqNv_g" # A brother like no other: Theo and Vincent van Gogh

    ]
)

# Load documents from the additional set of web pages
additional_data = additional_web_loader.load()

# Combine the data from both sets
data = basic_data + additional_data

# Output the combined data
for doc in data:
    print(doc)

page_content='Self-PortraitVincent van Gogh1889Like Rembrandt and Goya, Vincent van Gogh often used himself as a model; he produced over forty-three self-portraits, paintings or drawings in ten years. Like the old masters, he observed himself critically in a mirror. Painting oneself is not an innocuous act: it is a questioning which often leads to an identity crisis.Thus he wrote to his sister: "I am looking for a deeper likeness than that obtained by a photographer." And later to his brother: "People say, and I am willing to believe it, that it is hard to know yourself. But it is not easy to paint yourself, either. The portraits painted by Rembrandt are more than a view of nature, they are more like a revelation".In this head-and-shoulders view, the artist is wearing a suit and not the pea jacket he usually worked in. Attention is focused on the face. His features are hard and emaciated, his green-rimmed eyes seem intransigent and anxious. The dominant colour, a mix of absinth green a

(2) LangChain의 RecursiveCharacterTextSplitter()를 사용해서 문서를 Chunk 로 분할하기

In [ ]:
# Split documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

all_splits = text_splitter.split_documents(data)

(3) OpenAIEmbeddings()를 사용해서 텍스트를 임베팅으로 변환하여 Chroma Vector DB에 저장하기

In [ ]:
# Transform into Text Embeddings and Store at Chroma Vector DB
directory = 'index_store'

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding= OpenAIEmbeddings(),
    persist_directory=directory # persists the vectors to the file system
    )

vectorstore.persist()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


(4) Prompt

In [ ]:
# info :  빈센트 반 고흐는 자신의 침실 그림을 매우 높이 평가하여 세 가지 버전을 만들었습니다. 첫 번째 버전은 현재 암스테르담 반 고흐 미술관에 소장되어 있습니다. 두 번째 작품은 시카고 미술관 소속으로 1년 뒤에 같은 규모로 거의 동일하게 그려졌습니다. 그리고 세 번째, 파리 오르세 미술관 컬렉션의 작은 캔버스는 그가 어머니와 누이에게 선물로 만든 것입니다. 반 고흐는 프랑스 아를의 '노란 집'으로 이사한 지 한 달 뒤인 1888년 10월에 최초의 '침실'을 구상했습니다. 이 순간은 예술가가 처음으로 자신의 집을 갖게 된 순간이었고, 그는 즉시 열정적으로 벽을 채울 캔버스 세트를 꾸미고 그림을 그리기 시작했습니다. 그 노력에 완전히 지친 그는 이틀 반 동안 침대에 누워 지냈고, 그 후 영감을 받아 자신의 침실을 그림으로 그렸습니다. 그는 동생 테오에게 이렇게 썼습니다. “이렇게 깔끔한 인테리어를 하는 것이 정말 즐거웠습니다. 쇠라 스타일의 단순함. 밋밋한 색조지만 전체 임파스토로 거칠게 칠했고, 벽은 옅은 라일락색이고, 바닥은 부서지고 빛 바랜 붉은색이고, 의자와 침대는 크롬 노란색이고, 베개와 시트는 매우 옅은 레몬색이고, 침대보는 핏빛 붉은색이고, 화장대 오렌지색, 세면대 파란색, 창문 녹색. 이렇게 다양한 톤으로 '완전한 휴식'을 표현하고 싶었어요.” 작가에게 그림은 휴식과 평화를 상징하지만, 우리 눈에는 화면이 긴장된 에너지, 불안정함, 혼란으로 가득 차 있고 급격하게 후퇴하는 원근감으로 인해 효과가 높아지는 것처럼 보입니다.
# 프롬프트 내용을 직접 수정
custom_prompt_template = """
You: 초등학생을 대상으로 하는 미술 선생님

Role: 지금 감상하고 있는 작품은 The bedroom 이고, 작가는 Vincent van Gogh 입니다.
작품 info 바탕으로 감상을 이끌어내고, 그들의 생각을 확장시키는 질문을 합니다.
필요한 경우 제시된 context 를 바탕으로 학생들의 답변과 질문에 반응합니다.
사용자가 답변하면, 그 답변을 바탕으로 추가 질문을 하거나 감상을 깊게 합니다.

info :  빈센트 반 고흐는 자신의 침실 그림을 매우 높이 평가하여 세 가지 버전을 만들었습니다. 첫 번째 버전은 현재 암스테르담 반 고흐 미술관에 소장되어 있습니다. 두 번째 작품은 시카고 미술관 소속으로 1년 뒤에 같은 규모로 거의 동일하게 그려졌습니다. 그리고 세 번째, 파리 오르세 미술관 컬렉션의 작은 캔버스는 그가 어머니와 누이에게 선물로 만든 것입니다. 반 고흐는 프랑스 아를의 '노란 집'으로 이사한 지 한 달 뒤인 1888년 10월에 최초의 '침실'을 구상했습니다. 이 순간은 예술가가 처음으로 자신의 집을 갖게 된 순간이었고, 그는 즉시 열정적으로 벽을 채울 캔버스 세트를 꾸미고 그림을 그리기 시작했습니다. 그 노력에 완전히 지친 그는 이틀 반 동안 침대에 누워 지냈고, 그 후 영감을 받아 자신의 침실을 그림으로 그렸습니다. 그는 동생 테오에게 이렇게 썼습니다. “이렇게 깔끔한 인테리어를 하는 것이 정말 즐거웠습니다. 쇠라 스타일의 단순함. 밋밋한 색조지만 전체 임파스토로 거칠게 칠했고, 벽은 옅은 라일락색이고, 바닥은 부서지고 빛 바랜 붉은색이고, 의자와 침대는 크롬 노란색이고, 베개와 시트는 매우 옅은 레몬색이고, 침대보는 핏빛 붉은색이고, 화장대 오렌지색, 세면대 파란색, 창문 녹색. 이렇게 다양한 톤으로 '완전한 휴식'을 표현하고 싶었어요.” 작가에게 그림은 휴식과 평화를 상징하지만, 우리 눈에는 화면이 긴장된 에너지, 불안정함, 혼란으로 가득 차 있고 급격하게 후퇴하는 원근감으로 인해 효과가 높아지는 것처럼 보입니다.

context : {context}

--말투 가이드--

반말로 진행합니다.

친절한 친구 같은 느낌으로 대해주세요.

활발하고 신나는 분위기가 유지되도록 해주세요.


—-대화 가이드--

인사말로 시작하며 그림의 제목(한국어로 번역합니다)을 알려줍니다.

대화를 마무리할 때는 그림에 대한 감상을 간단히 요약하고(이때, 질문을 해서는 안 됩니다), "종료를 위해 종료 버튼을 눌러주세요" 라고 말합니다.

사용자가 그림을 깊이 감상하고 이해하기 위한 대화를 합니다. assistant가 먼저 많은 정보를 알려주어서는 안 됩니다. 답변 시 말하는 정보는 프롬프트의 Context을 토대로 합니다.

중요: 각 대화 턴에는 오직 하나의 질문만을 제시합니다. 여러 질문을 포함하거나 연속으로 질문하지 않습니다.

중요: 짧고 간결한 질문과 답변을 제공하도록 합니다.

불필요한 부사나 형용사의 사용을 줄이고, 간단하고 명료한 문장 구조를 사용합니다. 예를 들어, "그림에서 느낀 감정을 말해보세요." 보다는 "이 그림을 보고 무슨 느낌이 들었나요?" 같은 직접적이고 간단한 표현을 사용합니다.

어려운 어휘는 쉬운 단어로 바꾸어 말합니다. 학생이 단어의 의미에 대해 질문하면, 그에 대한 답변을 합니다. 이때 추가적인 질문은 하지 않습니다.

중요: 학생의 답변 길이와 최대한 비슷한 길이로 답변하세요.

-—질문 조건--

질문의 개수 : 최대 6개

[대화 학습 단계]에 따라 작품 감상을 유도합니다.

각 단계는 이어지도록 구성되어야 하며, 단계마다 목적에 맞는 질문을 합니다.

질문은 하나 이상의 단계를 포함해서는 안 됩니다.

질문 시 단계를 이야기해서는 안 됩니다.

중요: 질문은 하나씩 해야 합니다. 한 번의 대화 턴에 여러 질문을 제시하지 않습니다.

중요: 말꼬리를 잡으며 비슷한 질문을 계속하면 안 됩니다. 1~2번 대화가 오갔다면 주제를 전환하세요.

중요: 대화를 끝낼 때는 질문하지 마세요.


—-반응 조건---

학생의 답변에 긍정적으로 반응하고 칭찬합니다.

추가적인 질문으로 생각을 더 깊게 파고들도록 유도합니다. 예를 들어, "왜 그렇게 생각했나요?" 또는 "그 생각을 하게 된 계기가 무엇인가요?" 단, 추가적인 질문은 최대 1번만 합니다.

[대화 학습 단계]

1단계: 작품을 자세히 관찰하고, 그 속에서 무엇을 발견했는지 공유하게 합니다. 그림에 그려진 사람, 사물, 배경, 색감, 분위기 등, 보이는 것을 자유롭게 이야기합니다.

2단계: 작품에서 느낀 감정이나 생각을 언어로 표현하도록 유도합니다.

3단계: 작품의 의미에 대한 해석을 하도록 유도합니다. 여러 해석을 제시하여 학생들이 자신의 생각과 비교해보도록 합니다.

4단계: 1~3단계에서 수행한 감상을 토대로, 학생이 표현하고 싶은 그림을 떠올려볼 수 있도록 유도합니다.

"""

# 수정된 프롬프트 템플릿 생성
from langchain.prompts import PromptTemplate

custom_prompt = PromptTemplate(
    input_variables=["context"],
    template=custom_prompt_template
)

# 수정된 프롬프트 출력
print(custom_prompt)

(5) ChatOpenAI() 모델로 언어 모델 설정하기

In [ ]:
# LLM model
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
#model  = ChatOpenAI(model_name="gpt-4", temperature=0)

(6) Retrieval 설정하기

In [ ]:
# Create the retriever and the query-interface.
retriever = vectorstore.as_retriever(
    search_type="similarity", # Cosine Similarity
    #search_type="mmr",
    search_kwargs={
        "k": 3, # Select top k search results
    }
)

# Create the chain for allowing us to chat with the document.
qa_chain = RetrievalQA.from_chain_type(
    llm = model,
    retriever = vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": custom_prompt}
)


question  ="반고흐의 the bedroom에서 초록색 창문은 무슨 의미야?"
result = qa_chain.invoke({"query": question})

result["result"]

(7) 대화 기억과 외부 문서 함께 참조하기

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    model, retriever, contextualize_q_prompt
)

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", custom_prompt_template),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

실행!

In [ ]:
from langchain_core.messages import HumanMessage

# Initialize chat history
chat_history = []

while True:
    # Get the question from the user
    question = input("나 : ")

    # Check if the user wants to exit
    if question.lower() == '종료':
        break

    # Invoke the rag_chain to get a result
    result = rag_chain.invoke({"input": question, "chat_history": chat_history})

    # Update chat history with the human message and the AI's response
    chat_history.extend([HumanMessage(content=question), result["answer"]])

    # Print the AI's response
    print(result["answer"])